In [6]:
%load_ext autoreload
%autoreload 2
import sys 
import plotly.express as px
import plotly.graph_objects as go
import re
import time

import utils
from slurm import SlurmClient

# Slurm Queue

In [11]:
!sinfo

PARTITION AVAIL  TIMELIMIT  NODES  STATE NODELIST
cpu          up 7-00:00:00      1  down* cpu-c7i12xlarge-1
cpu          up 7-00:00:00      1   idle cpu-c7i12xlarge-0
q1*          up 7-00:00:00      1  comp* h100-st-p548xlarge-244
q1*          up 7-00:00:00      1  idle* h100-st-p548xlarge-285
q1*          up 7-00:00:00     23  down* h100-st-p548xlarge-[0,12,52,56-57,164,171-172,198,229-230,233-234,240-241,243,248,251,265,270,297,359,388]
q1*          up 7-00:00:00      4   comp h100-st-p548xlarge-[26,231,247,390]
q1*          up 7-00:00:00     13  drain h100-st-p548xlarge-[150-161,354]
q1*          up 7-00:00:00     33    mix h100-st-p548xlarge-[1,18-22,27-29,35-39,53-55,85-86,99-103,138,144,163,217,263,423-426]
q1*          up 7-00:00:00    352  alloc h100-st-p548xlarge-[3-11,13-17,23-25,30-34,40-51,58-84,87-97,104-137,139-143,145-149,162,165-170,173-197,199-215,218-226,228,235-239,245-246,249-250,252-262,264,266-269,271-284,286-296,298-353,355-358,360-387,389,391-422,431-436]


In [4]:
round(99.22/100, 3)

0.992

In [18]:
sl = SlurmClient()
q = sl.get_queue_summary()

print("Total:", q.NODES.sum())
q

Total: 479


,ACCOUNT,ST,NODES
0,ar-ai-hipri,PD,128
1,ar-ai-hipri,R,296
2,ar-ai-research-interns,R,4
3,ar-ai-voice-hipri,R,51


In [4]:
sl.get_recent_queue('all')

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS


In [42]:
sl.get_recent_queue('ar-ai-research-interns')

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS
27,9294,ar-ai-research-interns,jupyter,imzyc,R,2024-08-10T08:11:10,2-11:08:28,1,212908
28,8363,ar-ai-research-interns,bash,yfzhuang,R,2024-08-09T18:21:10,3-00:58:28,1,262708
29,8364,ar-ai-research-interns,bash,yfzhuang,R,2024-08-09T18:21:10,3-00:58:28,1,262708


In [5]:
sl.get_recent_queue('ar-ai-voice-hipri')

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS
64,12704,ar-ai-voice-hipri,bash,arashe,R,2024-08-19T23:35:13,12:50:54,1,46254
62,11486,ar-ai-voice-hipri,train2,liuyanga,R,2024-08-17T19:30:59,2-16:55:08,1,233708
63,11474,ar-ai-voice-hipri,train1,liuyanga,R,2024-08-17T18:50:01,2-17:36:06,1,236166


In [17]:
utils.get_bash_output("scancel 13008")

''

# Evict

In [21]:
# evic interns
interns = ['yfzhuang', 'jiuhai', 'imzyc']
sl = SlurmClient()
q = sl.get_queue()

q = q[q.USER.isin(interns)]
q = q[~q.ACCOUNT.isin(['ar-ai-research-interns', 'ar-ai-voice-hipri'])]

print(q[['JOBID', 'ACCOUNT', 'USER', 'NODES']])

    JOBID       ACCOUNT      USER  NODES
5   13854  ar-ai-midpri  yfzhuang      4
29  13833   ar-ai-hipri     imzyc      4
30  13844   ar-ai-hipri     imzyc     12
31  13845   ar-ai-hipri     imzyc     32
32  13846   ar-ai-hipri     imzyc     16


In [22]:
for j in list(q.JOBID):
    print(f"Cancelling job {j}")
    utils.get_bash_output(f"scancel {j}")

Cancelling job 13854
Cancelling job 13833
Cancelling job 13844
Cancelling job 13845
Cancelling job 13846


In [20]:
# evic me
# q = sl.get_recent_queue('ar-ai-hipri')
# q = q[q.USER == 'tranx']
# for j in q.JOBID:
#     if j != '12811':
#         utils.get_bash_output(f"scancel {j}")
#         print(f"Cancel {j}")

Cancel 13233
Cancel 13232
Cancel 13227
Cancel 13230
Cancel 13228
Cancel 13226
Cancel 13229
Cancel 13231
Cancel 13223
Cancel 13222
Cancel 13225
Cancel 13224
Cancel 13221
Cancel 13216
Cancel 13217
Cancel 13218
Cancel 13220
Cancel 13219
Cancel 13215


In [54]:
q = sl.get_queue_summary()
running = {}
for _, row in q.iterrows():
    if row.ST == 'R':
        running[row.ACCOUNT] = row.NODES
print(running)

quota = {'ar-ai-research-interns': 8,
         'ar-ai-voice-hipri': 50, 'ar-ai-hipri': 400}

evict_ids = {}
for acc in running:
    if acc not in quota:  # quota = 0
        print(acc)

{'all': 17, 'ar-ai-hipri': 274, 'ar-ai-midpri': 2, 'ar-ai-research-interns': 3, 'ar-ai-voice-hipri': 53}
all
ar-ai-midpri


In [30]:
q_summary = q.groupby(['ACCOUNT', 'ST']).aggregate(
    {'NODES': 'sum'}).reset_index()
q_summary

,ACCOUNT,ST,NODES
0,all,R,17
1,ar-ai-hipri,R,280
2,ar-ai-midpri,R,2
3,ar-ai-research-interns,R,3
4,ar-ai-voice-hipri,PD,94
5,ar-ai-voice-hipri,R,53


# Pretrain

In [48]:
MM9_CONF_DIR = "/fsx_0/user/tranx/experiments/llm_mm_aligner/stage1_mm9"
envvar = {
    "NUM": 7,
    "JSON_CONFIG": f"fbl_pretrain_MM9_70B_Llama31_336px_2nodes.json",
    # "CONDA_ENV": "aligner_v7",
}

sbatch_base = "/fsx_0/user/tranx/experiments/llm_mm_aligner/base_pretrain_MM9_70B_sbatch.sh"
sbatch_overwrite = {
    "job-name": "test",
    "nodes": 2,
    "ntasks": 2
}

MM9_CONF_DIR = "/fsx_0/user/tranx/experiments/llm_mm_aligner/stage1_mm9"
JSON_CONFIG = f"{MM9_CONF_DIR}/xxxfbl_pretrain_MM9_70B_Llama31_336px_2nodes.json"
ALIGNER_PARENT_DIR = "/fsx_0/user/tranx"

envvar_string = f"{JSON_CONFIG} {ALIGNER_PARENT_DIR}"

sbatch_vars_string = []
for k, v in sbatch_overwrite.items():
    sbatch_vars_string.append(f"--{k}={v}")
sbatch_vars_string = ' '.join(sbatch_vars_string)

# cmd = f'sbatch --parsable {sbatch_vars_string} {sbatch_base} {envvar_string}'
cmd = f'sbatch --parsable {sbatch_vars_string} {sbatch_base} a b'
print(cmd)

job_id = utils.get_bash_output(cmd, print_output=True)
job_id = int(job_id)

sbatch --parsable --job-name=test --nodes=2 --ntasks=2 /fsx_0/user/tranx/experiments/llm_mm_aligner/base_pretrain_MM9_70B_sbatch.sh a b
9659



In [29]:
!tail -f /fsx_0/user/tranx/output/slurm_logs/output_9402.txt

/var/spool/slurmd/job09402/slurm_script: line 23: conda: command not found
/var/spool/slurmd/job09402/slurm_script: line 24: conda: command not found
Using conda environment:
Error: CONDA_DEFAULT_ENV is not set to aligner_v7
^C


# Test trains

# Stage 2